In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, count, round, col, desc, rank
from pyspark.sql.window import Window
import os

def create_spark_session():
    spark = (
        SparkSession.builder
        .appName("Gold Layer - Soccer Analysis")
        .enableHiveSupport()
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.endpoint", os.getenv("MINIO_ENDPOINT"))
        .config("spark.hadoop.fs.s3a.access.key", os.getenv("MINIO_ROOT_USER"))
        .config("spark.hadoop.fs.s3a.secret.key", os.getenv("MINIO_ROOT_PASSWORD"))
        .getOrCreate()
    )
    return spark

spark = create_spark_session()

In [2]:
# Leia as tabelas Silver necessárias
df_match_silver = spark.read.format("delta").load("s3a://bucket-silver-zone/soccer/match/")
df_league_silver = spark.read.format("delta").load("s3a://bucket-silver-zone/soccer/league/")

In [4]:
# Tabela gold_goals_by_league
# Objetivo: Agregar gols por liga, país e temporada, com métricas como média de gols por partida.

df_goals_by_league = (
    df_match_silver
    .groupBy("country_name", "league_name", "season")
    .agg(
        sum("total_goals").alias("total_goals"),
        count("match_id").alias("total_matches"),
        round(sum("total_goals") / count("match_id"), 2).alias("avg_goals_per_match")
    )
    .orderBy(desc("avg_goals_per_match"))  # Ligas com mais gols
)

# Salvar como Delta Lake
(
    df_goals_by_league
    .write
    .format("delta")
    .mode("overwrite")
    .save("s3a://bucket-gold-zone/soccer/goals_by_league/")
)

In [8]:
# Tabela gold_goals_by_country
# Objetivo: Agregar gols por país (consolidando todas as ligas de um país).

df_goals_by_country = (
    df_match_silver
    .groupBy("country_name", "season")
    .agg(
        sum("total_goals").alias("total_goals"),
        count("match_id").alias("total_matches"),
        round(sum("total_goals") / count("match_id"), 2).alias("avg_goals_per_match")
    )
    .orderBy(desc("avg_goals_per_match"))  # Países com mais gols por partida
)

# Salvar como Delta Lake
(
    df_goals_by_country
    .write
    .format("delta")
    .mode("overwrite")
    .save("s3a://bucket-gold-zone/soccer/goals_by_country/")
)

In [7]:
# Tabela gold_top_leagues
# Objetivo: Destacar as top 5 ligas com mais gols em todas as temporadas.

window_spec = Window.partitionBy("season").orderBy(desc("avg_goals_per_match"))

df_top_leagues = (
    df_goals_by_league
    .withColumn("rank", rank().over(window_spec))
    .filter(col("rank") <= 5)  # Top 5
    .select("country_name", "league_name", "season", "avg_goals_per_match", "rank")
)

# Salvar como Delta Lake
(
    df_top_leagues
    .write
    .format("delta")
    .mode("overwrite")
    .save("s3a://bucket-gold-zone/soccer/top_leagues/")
)